# stage3_build_labelling
This notebook will use a __weak supervision package snorkel__ to generate labels. These labels will be used to train a simple classification model in the next step. 

The core task in this notebook is for me to write up a bunch of "Labelling functions", each of them simple, naive and noisy, they are purely based on my biased intuition of what diferent gendered customers may looks like. 

Snorkel will then try _"observing when and where these different labeling functions agree or disagree with one another, you can automatically learn—in unsupervised ways—when, where, and how much to trust each of them. You can thus learn their areas of expertise, and the overall level of expertise, so that when you combine their votes you end up with the highest quality label possible for each data point."_

This should work better than the opinionated analysis or heuristics that I can come up in short time. 

__The "Labelling Model" this produces at the end is a generalisation and statistical combination of my crude intuitions to the entire data set. It doesn't use all features (only the ones I wrote into the Labelling Functions) and it wouldn't generalise well into unseen data. Hence we still need another ML classification model to be trained on top of these labels.__

# Imports

In [1]:
from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis
from snorkel.labeling.model import LabelModel, MajorityLabelVoter


import pandas as pd
import numpy as np

/Users/muwang/opt/anaconda3/envs/env-challenge-ilikedata/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pathlib import Path
import sys

# a little path manipulation to load src/feature_engineering.py
root_path = str(Path('.').resolve().parent.absolute())

if root_path not in sys.path:
    sys.path.append(root_path)

from src.feature_engineering import fe_customer

# Load clean data and run feature engineering pipeline

In [3]:
clean_data_path = "../data/processed/clean_data.parquet"

clean_data = pd.read_parquet(clean_data_path)

In [4]:
features = fe_customer(clean_data)

In [5]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46279 entries, 0 to 46278
Data columns (total 44 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               46279 non-null  object 
 1   is_newsletter_subscriber  46279 non-null  float64
 2   cc_payments               46279 non-null  float64
 3   paypal_payments           46279 non-null  float64
 4   afterpay_payments         46279 non-null  float64
 5   apple_payments            46279 non-null  float64
 6   orders                    46279 non-null  float64
 7   items                     46279 non-null  float64
 8   cancels                   46279 non-null  float64
 9   returns                   46279 non-null  float64
 10  vouchers                  46279 non-null  float64
 11  female_items              46279 non-null  float64
 12  male_items                46279 non-null  float64
 13  unisex_items              46279 non-null  float64
 14  wapp_i

# Write labelling functions
I'll come up with as many ideas as I can while exploring the features

Setup some "constants"

In [6]:
# ABSTAIN is labelling function's way of saying "I dont' know"
# UNKNOWN is for when there's not enough data, for example customer hasn't bought anything yet
# Sorry, LGBTQI+ community, I don't have enough data or time to account for everyone

MALE = 0
FEMALE = 1
UNKNOWN = 2
ABSTAIN = -1

Explore the data:

In [7]:
with pd.option_context('display.max_columns', 999):
    display(features.sample(10))

,customer_id,is_newsletter_subscriber,cc_payments,paypal_payments,afterpay_payments,apple_payments,orders,items,cancels,returns,vouchers,female_items,male_items,unisex_items,wapp_items,wftw_items,mapp_items,wacc_items,macc_items,mftw_items,wspt_items,mspt_items,curvy_items,sacc_items,msite_orders,desktop_orders,android_orders,ios_orders,other_device_orders,work_orders,home_orders,parcelpoint_orders,other_collection_orders,redpen_discount_used,coupon_discount_applied,revenue,days_since_first_order,days_since_last_order,tenure_months,different_addresses,shipping_addresses,devices,average_discount_onoffer,average_discount_used
13843,c8610c546103cc2674ec90603bc8a2c9,0.0,1.0,0.0,0.0,0.0,1.000000,2.0000,0.0,0.000000,0.00,0.00,2.0000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,1.000000,0.00,0.0000,181.640000,1762.0,1762.0,1.0,0.0,1.0,1.0,0.0000,0.000000
20787,5e70b9e78e36c7e167b83e5dea781f88,1.0,1.0,0.0,0.0,0.0,1.000000,1.0000,0.0,0.000000,0.00,0.00,1.0000,0.0,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,1.000000,65.42,0.0000,43.620000,1741.0,1741.0,1.0,0.0,1.0,1.0,0.6000,0.599963
44335,cb7217cc814a67e0b3506ea0dedda899,0.0,1.0,0.0,0.0,0.0,1.000000,3.0000,0.0,0.000000,0.00,3.00,0.0000,0.0,3.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.0,0.0,1.000000,0.0,0.000000,40.83,0.0000,408.720000,699.0,699.0,1.0,0.0,1.0,1.0,0.0999,0.099927
5958,f8c33525428ddda5462fe13246728afd,0.0,0.0,1.0,0.0,0.0,1.000000,1.0000,0.0,0.000000,0.00,1.00,0.0000,0.0,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,1.000000,32.70,0.0000,0.000000,1785.0,1785.0,1.0,0.0,1.0,1.0,0.2999,0.299890
40226,340e1cfef8be2868e561182bf35c9006,0.0,1.0,0.0,0.0,0.0,2.000000,2.0000,0.0,0.000000,2.00,0.00,2.0000,0.0,0.000000,0.000000,0.000000,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.0,0.0,2.0,0.000000,0.0,0.000000,29.09,39.6100,185.740000,407.0,406.0,1.0,0.0,1.0,2.0,0.1000,0.254993
10162,b3b4a13d8499890cd492bd7830bf5d79,0.0,0.0,1.0,0.0,0.0,1.000000,2.0000,0.0,1.000000,0.00,2.00,0.0000,0.0,0.000000,2.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,1.000000,54.48,0.0000,108.960000,1765.0,1765.0,1.0,0.0,1.0,1.0,0.3000,0.300000
35620,537efb35d6235942045b915eeb69a3a2,0.0,1.0,1.0,0.0,0.0,0.062500,0.1875,0.0,0.041667,0.00,0.00,0.1875,0.0,0.000000,0.000000,0.104167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,0.041667,0.000000,0.0,0.0,0.0,0.041667,0.0,0.020833,0.00,0.0000,43.192917,1424.0,9.0,48.0,0.0,2.0,2.0,0.0000,0.000000
12640,fee92c4a61f9c28a3735e6081f1e7f1d,0.0,0.0,1.0,0.0,0.0,1.000000,2.0000,0.0,0.000000,0.00,2.00,0.0000,0.0,2.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,1.000000,0.00,0.0000,236.180000,919.0,919.0,1.0,0.0,1.0,1.0,0.0000,0.000000
6922,5efb89ca2842267d1082775b429d26f4,0.0,0.0,1.0,0.0,0.0,2.000000,2.0000,0.0,0.000000,0.00,2.00,0.0000,0.0,2.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,2.000000,0.00,0.0000,131.720000,1833.0,1808.0,1.0,0.0,1.0,2.0,0.0000,0.000000
43467,d921eaf378e552163b5ebaa91ebc0f73,1.0,1.0,0.0,0.0,0.0,0.666667,1.7500,0.0,0.250000,0.25,1.75,0.0000,0.0,1.083333,0.666667,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.500000,0.083333,0.0,0.0,0.0,0.583333,0.0,0.083333,121.29,13.6075,83.387500,460.0,122.0,12.0,0.0,1.0,2.0,0.5559,0.626003


In [8]:
features['items'].describe()

count    46279.000000
mean         1.629684
std          2.770734
min          0.029851
25%          0.619048
50%          1.000000
75%          2.000000
max        232.000000
Name: items, dtype: float64

In [9]:
@labeling_function()
def bought_male_item(x):
    return (
        MALE 
        if x.male_items > 0
        else ABSTAIN
    )

@labeling_function()
def bought_more_male_item(x):
    return (
        MALE 
        if x.male_items > x.female_items
        else ABSTAIN
    )

@labeling_function()
def bought_female_item(x):
    return (
        FEMALE 
        if x.female_items > 0
        else ABSTAIN
    )

@labeling_function()
def bought_more_female_item(x):
    return (
        FEMALE 
        if x.female_items > x.male_items
        else ABSTAIN
    )

@labeling_function()
def no_purchase(x):
    return (
        UNKNOWN
        if x['items'] == 0
        else ABSTAIN
    )

# Evaluate labelling functions on feature set - first pass

In [10]:
labelling_functions = [bought_male_item, bought_more_male_item, 
                       bought_female_item, bought_more_female_item, no_purchase]
applier = PandasLFApplier(lfs=labelling_functions)
L_train = applier.apply(df=features)

100%|██████████████████████████████| 46279/46279 [00:02<00:00, 16871.53it/s]


In [11]:
coverage_array = (pd.DataFrame(L_train) != ABSTAIN).mean(axis=0)

In [12]:
LFAnalysis(L=L_train, lfs=labelling_functions).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
bought_male_item,0,[0],0.371356,0.371356,0.162838
bought_more_male_item,1,[0],0.246354,0.246354,0.037836
bought_female_item,2,[1],0.745975,0.745975,0.162838
bought_more_female_item,3,[1],0.689600,0.689600,0.106463
no_purchase,4,[],0.000000,0.000000,0.000000


Oops, no_purchase() has 0 coverage, meaning all customers in this dataset has bought something, anyway...

# Keep writing more labelling functions

In [13]:
@labeling_function()
def unisex_only(x):
    return (
        UNKNOWN 
        if x.male_items == 0 
            and x.female_items == 0
            and x.unisex_items > 0
        else ABSTAIN
    )

@labeling_function()
def more_than_one_female_categories(x):
    return (
        FEMALE
        if np.sum([
                x.wapp_items > 0, 
                  x.wftw_items > 0, 
                  x.wacc_items > 0,
                  x.wspt_items > 0,
        ]) > 1.001
        else ABSTAIN
    )

@labeling_function()
def more_than_one_male_categories(x):
    return (
        MALE
        if np.sum([x.mapp_items > 0, 
                  x.macc_items > 0, 
                  x.mftw_items > 0,
                  x.mspt_items > 0]
                 ) > 1.001
        else ABSTAIN
    )

# Evaluate labelling functions on feature set - second pass

In [14]:
labelling_functions = [bought_male_item, bought_more_male_item, 
                       bought_female_item, bought_more_female_item, no_purchase,
                      unisex_only, more_than_one_female_categories, more_than_one_male_categories
                      ]
applier = PandasLFApplier(lfs=labelling_functions)
L_train = applier.apply(df=features)

100%|███████████████████████████████| 46279/46279 [00:06<00:00, 6809.67it/s]


In [15]:
coverage_array = (pd.DataFrame(L_train) != ABSTAIN).mean(axis=0)

In [16]:
LFAnalysis(L=L_train, lfs=labelling_functions).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
bought_male_item,0,[0],0.371356,0.371356,0.163011
bought_more_male_item,1,[0],0.246354,0.246354,0.038009
bought_female_item,2,[1],0.745975,0.745975,0.170877
bought_more_female_item,3,[1],0.689600,0.689600,0.114501
no_purchase,4,[],0.000000,0.000000,0.000000
unisex_only,5,[2],0.045507,0.000324,0.000324
more_than_one_female_categories,6,[1],0.319843,0.319843,0.108710
more_than_one_male_categories,7,[0],0.139113,0.139113,0.088204


# Examine conflicts
Now that we are starting to see more conflicts between different LFs, it'd be good to dive in and see what's happening. It should give me more ideas. 

In [17]:
# how many different votes other than ABSTAIN each example got:
multi_votes = pd.DataFrame(L_train).replace(-1, np.NaN)
multi_votes.columns = [x.name for x in labelling_functions]
multi_votes

,bought_male_item,bought_more_male_item,bought_female_item,bought_more_female_item,no_purchase,unisex_only,more_than_one_female_categories,more_than_one_male_categories
0,0.0,NaN,1.0,1.0,NaN,NaN,1.0,0.0
1,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN
2,0.0,NaN,1.0,1.0,NaN,NaN,1.0,0.0
3,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
46274,0.0,NaN,1.0,1.0,NaN,NaN,1.0,0.0
46275,0.0,NaN,1.0,1.0,NaN,NaN,1.0,0.0
46276,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
46277,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN


In [18]:
multi_votes_filter = (multi_votes.apply(lambda s: s.nunique(), axis=1) > 1)

In [19]:
with pd.option_context('display.max_columns', 999):
    display(
        pd.concat([
            multi_votes.loc[multi_votes_filter,:],
            features.loc[multi_votes_filter,:]
        ], axis=1).sample(10)
    )

,bought_male_item,bought_more_male_item,bought_female_item,bought_more_female_item,no_purchase,unisex_only,more_than_one_female_categories,more_than_one_male_categories,customer_id,is_newsletter_subscriber,cc_payments,paypal_payments,afterpay_payments,apple_payments,orders,items,cancels,returns,vouchers,female_items,male_items,unisex_items,wapp_items,wftw_items,mapp_items,wacc_items,macc_items,mftw_items,wspt_items,mspt_items,curvy_items,sacc_items,msite_orders,desktop_orders,android_orders,ios_orders,other_device_orders,work_orders,home_orders,parcelpoint_orders,other_collection_orders,redpen_discount_used,coupon_discount_applied,revenue,days_since_first_order,days_since_last_order,tenure_months,different_addresses,shipping_addresses,devices,average_discount_onoffer,average_discount_used
34233,0.0,0.0,1.0,NaN,NaN,NaN,1.0,0.0,92c10b686417013dbfdc450bd8a715a9,1.0,1.0,1.0,0.0,0.0,0.166667,0.366667,0.000000,0.033333,0.066667,0.050000,0.283333,0.033333,0.033333,0.000000,0.266667,0.016667,0.016667,0.016667,0.000000,0.033333,0.0,0.00000,0.050000,0.050000,0.066667,0.000000,0.0,0.0,0.016667,0.0,0.150000,13.201167,3.468000,35.260167,1917.0,117.0,60.0,0.0,3.0,3.0,0.2477,0.305408
19105,0.0,NaN,1.0,1.0,NaN,NaN,1.0,0.0,801aec091fbd42ff7b1d8f142e5b062e,1.0,1.0,1.0,0.0,0.0,0.229508,0.491803,0.016393,0.065574,0.163934,0.213115,0.065574,0.213115,0.081967,0.131148,0.016393,0.098361,0.098361,0.016393,0.000000,0.016393,0.0,0.04918,0.081967,0.114754,0.000000,0.032787,0.0,0.0,0.049180,0.0,0.180328,21.431803,11.784098,50.514098,1965.0,160.0,61.0,0.0,2.0,3.0,0.2175,0.361242
18952,0.0,NaN,1.0,1.0,NaN,NaN,1.0,0.0,e5150a3ec3bf659fe1bac6b31fca4c8a,0.0,1.0,1.0,0.0,0.0,0.194444,0.333333,0.000000,0.055556,0.083333,0.194444,0.138889,0.000000,0.000000,0.111111,0.027778,0.000000,0.000000,0.027778,0.083333,0.000000,0.0,0.00000,0.055556,0.138889,0.000000,0.000000,0.0,0.0,0.027778,0.0,0.166667,3.353889,1.261667,33.255833,1741.0,664.0,36.0,0.0,2.0,3.0,0.1179,0.144782
25553,NaN,NaN,1.0,1.0,NaN,NaN,1.0,0.0,e4bb1a9293ecc59d9a9c803386c92cdd,0.0,0.0,0.0,1.0,0.0,0.666667,0.833333,0.000000,0.000000,0.166667,0.500000,0.000000,0.333333,0.333333,0.000000,0.000000,0.166667,0.166667,0.333333,0.000000,0.000000,0.0,0.00000,0.333333,0.166667,0.000000,0.166667,0.0,0.0,0.666667,0.0,0.000000,0.000000,2.723333,116.501667,187.0,25.0,6.0,0.0,2.0,3.0,0.0000,0.042831
16062,0.0,0.0,1.0,NaN,NaN,NaN,NaN,0.0,547032dffbf4a7e4af2fe50deecece70,0.0,1.0,1.0,0.0,0.0,2.800000,4.000000,0.000000,1.200000,0.400000,1.000000,3.000000,0.000000,1.000000,0.000000,2.400000,0.000000,0.000000,0.600000,0.000000,0.000000,0.0,0.00000,0.000000,2.800000,0.000000,0.000000,0.0,0.0,0.000000,0.0,2.800000,309.628000,8.174000,565.440000,1756.0,1619.0,5.0,0.0,1.0,1.0,0.3430,0.362285
27363,0.0,0.0,1.0,NaN,NaN,NaN,1.0,0.0,fcb68d64131e129c054163d23d779c9c,1.0,1.0,0.0,0.0,0.0,4.000000,13.000000,0.000000,2.000000,1.000000,2.000000,11.000000,0.000000,1.000000,0.000000,5.000000,0.000000,0.000000,3.000000,1.000000,0.000000,0.0,0.00000,0.000000,4.000000,0.000000,0.000000,0.0,0.0,4.000000,0.0,0.000000,4288.840000,82.620000,4433.520000,489.0,465.0,1.0,0.0,1.0,1.0,0.4868,0.496897
11761,0.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,2f9201b9c255deb080a9082f8f271ada,1.0,1.0,0.0,0.0,0.0,1.169492,1.983051,0.000000,0.000000,0.220339,1.949153,0.033898,0.000000,1.864407,0.000000,0.000000,0.000000,0.000000,0.000000,0.016949,0.000000,0.0,0.00000,0.000000,1.169492,0.000000,0.000000,0.0,0.0,0.220339,0.0,0.949153,130.870000,7.210169,254.909153,1782.0,34.0,59.0,0.0,1.0,1.0,0.2709,0.285176
19173,0.0,0.0,1.0,NaN,NaN,NaN,1.0,NaN,b2e7464257816bfdfc1e5fa75a7580bd,0.0,1.0,0.0,0.0,0.0,0.148936,0.595745,0.000000,0.063830,0.042553,0.063830,0.531915,0.000000,0.042553,0.021277,0.510638,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.106383,0.042553,0.000000,0.000000,0.0,0.0,0.063830,0.0,0.085106,8.702128,11.541064,101.874894,1738.0,350.0,47.0,0.0,3.0,3.0,0.0385,0.152497
18077,0.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,daaecc801c861726bd8b13812c2477ac,1.0,1.0,1.0,0.0,0.0

Here are some observations:
- customers who bought equal male and female items
- customers who bought unisex only but also bought men's footwear (is men's footwear unisex?)
- found some bugs in the LFs
- customers who bought both male and female, but lean heavily into one side

These give me a few more ideas to try

# The third round of LFs and evaluation

In [20]:
@labeling_function()
def two_times_more_female_item(x):
    return (
        FEMALE 
        if x.female_items >= 2.0 * x.male_items
        else ABSTAIN
    )

@labeling_function()
def two_times_more_male_item(x):
    return (
        MALE 
        if x.male_items >= 2.0 * x.female_items
        else ABSTAIN
    )

@labeling_function()
def roughtly_equal_male_female_item(x):
    """My experience tells me this is still more likely female customer :) """
    return (
        FEMALE 
        if x.female_items >= 0.9 * x.male_items
            and x.female_items <= 1.1 * x.male_items
        else ABSTAIN
    )

@labeling_function()
def equal_or_more_female_categories(x):
    return (
        FEMALE
        if np.sum([
                x.wapp_items > 0, 
                  x.wftw_items > 0, 
                  x.wacc_items > 0,
                  x.wspt_items > 0,
        ]) >= np.sum([x.mapp_items > 0, 
                  x.macc_items > 0, 
                  x.mftw_items > 0,
                  x.mspt_items > 0]
                 )
        else ABSTAIN
    )

@labeling_function()
def more_male_categories(x):
    return (
        MALE
        if np.sum([x.mapp_items > 0, 
                  x.macc_items > 0, 
                  x.mftw_items > 0,
                  x.mspt_items > 0]
                 ) > np.sum([
                x.wapp_items > 0, 
                  x.wftw_items > 0, 
                  x.wacc_items > 0,
                  x.wspt_items > 0,
        ])
        else ABSTAIN
    )

@labeling_function()
def lot_more_female_item(x):
    return (
        FEMALE 
        if x.female_items >= 3.0 * x.male_items
        else ABSTAIN
    )

@labeling_function()
def lot_more_male_item(x):
    return (
        MALE 
        if x.male_items >= 3.0 * x.female_items
        else ABSTAIN
    )


@labeling_function()
def ten_times_more_female_item(x):
    return (
        FEMALE 
        if x.female_items >= 10.0 * x.male_items
        else ABSTAIN
    )

@labeling_function()
def ten_times_more_male_item(x):
    return (
        MALE 
        if x.male_items >= 10.0 * x.female_items
        else ABSTAIN
    )

@labeling_function()
def no_male_categories(x):
    return (
        FEMALE
        if np.sum([x.mapp_items > 0, 
                  x.macc_items > 0, 
                  x.mftw_items > 0,
                  x.mspt_items > 0]
                 ) == 0
        else ABSTAIN
    )

@labeling_function()
def no_female_categories(x):
    return (
        MALE
        if np.sum([
                x.wapp_items > 0, 
                  x.wftw_items > 0, 
                  x.wacc_items > 0,
                  x.wspt_items > 0,
        ]) == 0
        else ABSTAIN
    )

In [21]:
labelling_functions = [bought_male_item, bought_more_male_item, 
                       bought_female_item, bought_more_female_item, no_purchase,
                      unisex_only, more_than_one_female_categories, more_than_one_male_categories,
                       two_times_more_female_item, two_times_more_male_item, roughtly_equal_male_female_item,
                       equal_or_more_female_categories, more_male_categories,
                       lot_more_female_item, lot_more_male_item,
                       ten_times_more_female_item, ten_times_more_male_item,
                       no_male_categories,no_female_categories
                      ]
applier = PandasLFApplier(lfs=labelling_functions)
L_train = applier.apply(df=features)

100%|███████████████████████████████| 46279/46279 [00:20<00:00, 2292.95it/s]


In [22]:
coverage_array = (pd.DataFrame(L_train) != ABSTAIN).mean(axis=0)

In [23]:
LFAnalysis(L=L_train, lfs=labelling_functions).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
bought_male_item,0,[0],0.371356,0.371356,0.202446
bought_more_male_item,1,[0],0.246354,0.246354,0.077443
bought_female_item,2,[1],0.745975,0.745975,0.186197
bought_more_female_item,3,[1],0.689600,0.689600,0.129821
no_purchase,4,[],0.000000,0.000000,0.000000
unisex_only,5,[2],0.045507,0.045507,0.045507
more_than_one_female_categories,6,[1],0.319843,0.319843,0.108710
more_than_one_male_categories,7,[0],0.139113,0.139113,0.088204
two_times_more_female_item,8,[1],0.724411,0.724411,0.164632
two_times_more_male_item,9,[0],0.283347,0.283347,0.114436


# Combine all LFs to generate a statistical labelling model
It can get much more sophisticated than that, for example with expert inputs from SMEs. But I'm clearly no SME in this area, thinking on it even more might have diminished return. 

I think that's good enough given the time spent

Time to generate Labels!

In [24]:
L_train.shape

(46279, 19)

## First build a simple majority vote labelling model

In [25]:
majority_model = MajorityLabelVoter(cardinality=3)
preds = majority_model.predict(L=L_train)

In [26]:
label_dict = {0:'MALE', 1:'FEMALE', 2:'UNKNOWN', -1:'ABSTAIN'}
pd.Series(preds).replace(label_dict).value_counts()

FEMALE     34143
MALE       11765
ABSTAIN      371
dtype: int64

Not too bad, it covered the vast majority of the data

Still I'll check the examples where all LFs abstained

In [27]:
with pd.option_context('display.max_columns', 999):
    display(
        features.loc[preds == -1]
    )

,customer_id,is_newsletter_subscriber,cc_payments,paypal_payments,afterpay_payments,apple_payments,orders,items,cancels,returns,vouchers,female_items,male_items,unisex_items,wapp_items,wftw_items,mapp_items,wacc_items,macc_items,mftw_items,wspt_items,mspt_items,curvy_items,sacc_items,msite_orders,desktop_orders,android_orders,ios_orders,other_device_orders,work_orders,home_orders,parcelpoint_orders,other_collection_orders,redpen_discount_used,coupon_discount_applied,revenue,days_since_first_order,days_since_last_order,tenure_months,different_addresses,shipping_addresses,devices,average_discount_onoffer,average_discount_used
165,86774735323382706c516590772d1972,1.0,1.0,0.0,0.0,0.0,0.285714,0.619048,0.0,0.000000,0.047619,0.238095,0.285714,0.095238,0.000000,0.285714,0.000000,0.047619,0.047619,0.285714,0.000000,0.00,0.0,0.00,0.047619,0.238095,0.0,0.00,0.0,0.0,0.000000,0.0,0.285714,42.518571,0.000000,115.843810,1972.0,1367.0,21.0,0.0,2.0,2.0,0.3465,0.346467
241,57fc4e2f27066a39532751686c43ec2b,1.0,1.0,0.0,0.0,0.0,2.500000,19.500000,0.0,0.000000,0.000000,6.000000,10.500000,3.000000,0.000000,6.000000,0.000000,0.000000,0.000000,7.500000,0.000000,0.00,0.0,2.00,0.000000,2.500000,0.0,0.00,0.0,0.0,0.000000,0.0,2.500000,0.000000,0.000000,3273.130000,2011.0,1951.0,2.0,0.0,1.0,1.0,0.0000,0.000000
282,8533224f598d39f203964439c2b75280,1.0,1.0,1.0,0.0,0.0,0.133333,0.150000,0.0,0.000000,0.033333,0.050000,0.100000,0.000000,0.016667,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.10,0.0,0.00,0.000000,0.133333,0.0,0.00,0.0,0.0,0.016667,0.0,0.116667,10.120000,1.798167,5.485667,2064.0,278.0,60.0,0.0,3.0,1.0,0.4415,0.510553
502,a1b1c39eda866cd14190e5b4b406e790,1.0,1.0,0.0,0.0,0.0,0.434783,0.826087,0.0,0.086957,0.043478,0.130435,0.565217,0.130435,0.086957,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.391304,0.043478,0.0,0.00,0.0,0.0,0.000000,0.0,0.434783,22.496957,0.000000,38.599130,2004.0,1320.0,23.0,0.0,4.0,2.0,0.3859,0.385893
742,4b2a2e97f60c8dbdd72a41da0ff2fe16,0.0,0.0,1.0,0.0,0.0,2.000000,5.000000,0.0,3.000000,0.000000,2.000000,3.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.00,0.0,0.00,0.000000,2.000000,0.0,0.00,0.0,0.0,0.000000,0.0,2.000000,0.000000,0.000000,926.830000,2016.0,2016.0,1.0,0.0,1.0,1.0,0.0000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45729,3e818b0a1b2d0590eadbbeeede9f8409,1.0,0.0,1.0,0.0,0.0,1.000000,4.250000,0.0,0.500000,0.000000,2.750000,1.000000,0.500000,2.750000,0.000000,1.000000,0.000000,0.000000,0.500000,0.000000,0.00,0.0,0.00,0.500000,0.500000,0.0,0.00,0.0,0.0,1.000000,0.0,0.000000,626.160000,0.000000,1085.400000,261.0,162.0,4.0,0.0,1.0,2.0,0.3523,0.352329
45781,22268bb02095239eeca22b4248ec6eb1,1.0,1.0,1.0,0.0,0.0,0.550000,1.200000,0.0,0.200000,0.000000,0.650000,0.450000,0.100000,0.350000,0.300000,0.050000,0.000000,0.000000,0.150000,0.000000,0.05,0.0,0.05,0.100000,0.000000,0.0,0.45,0.0,0.0,0.550000,0.0,0.000000,16.391000,0.000000,121.933500,787.0,197.0,20.0,0.0,1.0,1.0,0.0984,0.098399
45840,f42fc634ab794767015fdd183546df63,0.0,1.0,0.0,0.0,0.0,0.300000,0.400000,0.0,0.000000,0.100000,0.200000,0.100000,0.100000,0.000000,0.000000,0.000000,0.200000,0.200000,0.000000,0.000000,0.10,0.0,0.00,0.000000,0.300000,0.0,0.00,0.0,0.0,0.300000,0.0,0.000000,0.000000,0.817000,19.161000,659.0,378.0,10.0,0.0,1.0,1.0,0.0000,0.024985
46089,1a8a061c17524ea5e2933bff04805592,0.0,0.0,1.0,0.0,0.0,1.000000,1.500000,0.0,0.000000,0.500000,0.500000,1.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.000000,1.000000,0.0,0.00,0.0,0.0,0.000000,0.0,1.000000,31.800000,9.090000,77.170000,1879.0,1837.0,2.0,0.0,1.0,1.0,0.1000,0.233557


In [28]:
L_train[14,:]

array([ 0, -1,  1,  1, -1, -1,  1,  0, -1, -1, -1,  1, -1, -1, -1, -1, -1,
       -1, -1])

In [29]:
L_train[32,:]

array([ 0, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1, -1,
       -1, -1])

Looks like we got some ties in the voting, this is fine, a more sophisticated labelling model should resolve the ties with weighted confidenc in each LF

## Build a probabilistic or confidence-weighted labelling model

In [30]:
label_model = LabelModel(cardinality=3)
label_model.fit(L_train=L_train, n_epochs=1000, log_freq=100, seed=273)

INFO:root:Computing O...
INFO:root:Estimating \mu...
  0%|                                           | 0/1000 [00:00<?, ?epoch/s]INFO:root:[0 epochs]: TRAIN:[loss=23.521]
INFO:root:[100 epochs]: TRAIN:[loss=0.121]
 16%|████▉                          | 158/1000 [00:00<00:00, 1570.20epoch/s]INFO:root:[200 epochs]: TRAIN:[loss=0.032]
INFO:root:[300 epochs]: TRAIN:[loss=0.031]
 49%|███████████████▏               | 488/1000 [00:00<00:00, 1579.55epoch/s]INFO:root:[500 epochs]: TRAIN:[loss=0.031]
INFO:root:[600 epochs]: TRAIN:[loss=0.031]
 80%|████████████████████████▊      | 800/1000 [00:00<00:00, 1496.62epoch/s]INFO:root:[800 epochs]: TRAIN:[loss=0.031]
INFO:root:[900 epochs]: TRAIN:[loss=0.031]
100%|██████████████████████████████| 1000/1000 [00:00<00:00, 1522.53epoch/s]
INFO:root:Finished Training


The loss didn't decrease further, 1000 epochs is fine

In [31]:
adv_preds = label_model.predict(L_train)

In [32]:
pd.Series(adv_preds).replace(label_dict).value_counts()

FEMALE     25906
MALE       13000
UNKNOWN     7373
dtype: int64

Very good! More FEMALE, less MALE and some UNKNOWN as well.

## Examine disagreement between voting and probabilistic model

In [33]:
diff_preds = (preds != adv_preds)

In [34]:
pd.Series(diff_preds).value_counts()

False    36695
True      9584
dtype: int64

In [35]:
pd.concat([
            pd.Series(preds).rename('majority_model'), 
            pd.Series(adv_preds).rename('proba_model'),
        ], axis=1)\
[lambda df: df.majority_model != df.proba_model]\
.groupby(['majority_model','proba_model']).size()

majority_model  proba_model
-1              0                70
                2               301
 0              2               976
 1              0              2141
                2              6096
dtype: int64

In [36]:
with pd.option_context('display.max_columns', 999):
    display(
        pd.concat([
            pd.Series(preds).rename('majority_model'), 
            pd.Series(adv_preds).rename('proba_model'),
            features
        ], axis=1).loc[diff_preds].sample(10)
    )

,majority_model,proba_model,customer_id,is_newsletter_subscriber,cc_payments,paypal_payments,afterpay_payments,apple_payments,orders,items,cancels,returns,vouchers,female_items,male_items,unisex_items,wapp_items,wftw_items,mapp_items,wacc_items,macc_items,mftw_items,wspt_items,mspt_items,curvy_items,sacc_items,msite_orders,desktop_orders,android_orders,ios_orders,other_device_orders,work_orders,home_orders,parcelpoint_orders,other_collection_orders,redpen_discount_used,coupon_discount_applied,revenue,days_since_first_order,days_since_last_order,tenure_months,different_addresses,shipping_addresses,devices,average_discount_onoffer,average_discount_used
42913,1,2,4937847321f5a26bc241b52674d4e106,1.0,1.0,0.0,0.0,0.0,1.333333,4.166667,0.000000,2.222222,0.638889,4.111111,0.000000,0.055556,3.361111,0.527778,0.00000,0.166667,0.166667,0.027778,0.055556,0.000000,0.000000,0.0,0.027778,1.305556,0.000000,0.000000,0.0,0.0,1.166667,0.00000,0.166667,368.065278,66.925833,865.618056,1083.0,27.0,36.0,0.0,1.0,2.0,0.2575,0.311997
14923,1,2,20a522e1b0dba5ef380d38ede7044998,0.0,1.0,0.0,0.0,0.0,0.714286,1.285714,0.000000,0.428571,0.000000,1.142857,0.142857,0.000000,0.000000,0.571429,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.714286,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.714286,89.268571,0.000000,104.761429,1758.0,1559.0,7.0,1.0,2.0,1.0,0.3360,0.335960
8234,1,0,b481039974514f09794960515b59077e,0.0,1.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,1.000000,100.000000,0.000000,53.640000,1816.0,1816.0,1.0,0.0,1.0,1.0,0.6509,0.650915
42828,1,2,a6b64a75ef4c468e5e39273b34b1dba3,0.0,1.0,1.0,0.0,0.0,0.687500,1.437500,0.062500,0.250000,0.250000,1.312500,0.125000,0.000000,1.000000,0.187500,0.12500,0.062500,0.062500,0.000000,0.062500,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.437500,0.0,0.0,0.687500,0.00000,0.000000,45.780625,29.956875,125.417500,488.0,15.0,16.0,0.0,3.0,2.0,0.1835,0.360897
19787,1,2,25422a038760db71013276f4e49eb4d0,1.0,1.0,1.0,0.0,0.0,0.196429,0.214286,0.000000,0.071429,0.107143,0.196429,0.017857,0.000000,0.035714,0.089286,0.00000,0.035714,0.035714,0.000000,0.035714,0.017857,0.000000,0.0,0.160714,0.035714,0.000000,0.000000,0.0,0.0,0.071429,0.00000,0.125000,1.870536,3.424107,19.761071,1741.0,72.0,56.0,1.0,3.0,3.0,0.0791,0.191958
16631,-1,2,bcc8fd0a6ca748daf1135312021e399d,0.0,0.0,1.0,0.0,0.0,0.210526,0.236842,0.000000,0.131579,0.052632,0.157895,0.078947,0.000000,0.000000,0.157895,0.00000,0.000000,0.000000,0.026316,0.000000,0.052632,0.000000,0.0,0.026316,0.184211,0.000000,0.000000,0.0,0.0,0.026316,0.00000,0.184211,15.736579,4.306316,10.089737,1750.0,632.0,38.0,0.0,1.0,2.0,0.3727,0.524271
9450,1,2,3e0bcf19c2fcb9f6a117efe956bff3fe,0.0,1.0,1.0,0.0,0.0,0.244444,0.466667,0.000000,0.111111,0.000000,0.444444,0.022222,0.000000,0.066667,0.177778,0.00000,0.044444,0.044444,0.000000,0.000000,0.000000,0.155556,0.0,0.044444,0.133333,0.000000,0.066667,0.0,0.0,0.133333,0.00000,0.111111,9.618000,0.000000,88.052444,1763.0,418.0,45.0,0.0,2.0,2.0,0.1025,0.102487
22384,1,2,8f76e672013f76f8db12352309c9e61a,0.0,1.0,0.0,0.0,0.0,0.035714,0.053571,0.000000,0.035714,0.000000,0.035714,0.000000,0.017857,0.000000,0.000000,0.00000,0.035714,0.035714,0.017857,0.000000,0.000000,0.000000,0.0,0.000000,0.035714,0.000000,0.000000,0.0,0.0,0.017857,0.00000,0.017857,0.000000,0.000000,4.025000,1738.0,65.0,56.0,1.0,2.0,1.0,0.0000,0.000000
13592,1,2,01d096a0b502b8e4969c13f0a301eaf6,1.0,1.0,1.0,0.0,0.0,0.777778,1.962963,0.037037,0.814815,0.444444,1.481481,0.370370,0.111111,1.259259,0.111111,0.37037,0.037037,0.037037,0.111111,0.074074,0.000000,0.000000,0.0,0.333333,0.444444,0.000000,0.000000,0.0,0.0,0.222222,0.37037,0.185185,74.298148,150.793704,276.865926,916.0,117.0,27.0,1.0,10.0,3.0,0.1212,0.368792
6411,1,2,a8dee84d047d9f8764ddf10718b60010,1.0,1.0,1.0,1.0,0.0,0.1

# Save labels and final thoughts
When the two models disagree, I actually like the Majority voting model better, despite its simplicity. This exposes potential issue in the way that I wrote the LFs or my personal bias. I could further tweak the LFs or try validating against a ground truth test set. But I will leave it here because I don't want to spend too much time. 

In [40]:
training_set = pd.concat([
            pd.Series(preds).rename('gender').replace(label_dict).replace('ABSTAIN', 'UNKNOWN'),
            features
        ], axis=1)

training_set

,gender,customer_id,is_newsletter_subscriber,cc_payments,paypal_payments,afterpay_payments,apple_payments,orders,items,cancels,...,coupon_discount_applied,revenue,days_since_first_order,days_since_last_order,tenure_months,different_addresses,shipping_addresses,devices,average_discount_onoffer,average_discount_used
0,FEMALE,64f7d7dd7a59bba7168cc9c960a5c60e,0.0,1.0,0.0,0.0,0.0,0.354167,1.041667,0.000000,...,5.180208,144.715417,2091.0,653.0,48.0,0.0,4.0,1.0,0.3364,0.358448
1,FEMALE,fa7c64efd5c037ff2abcce571f9c1712,1.0,0.0,1.0,0.0,0.0,0.188406,0.376812,0.000000,...,0.000000,77.235942,2082.0,22.0,69.0,0.0,4.0,2.0,0.1404,0.140410
2,FEMALE,18923c9361f27583d2320951435e4888,1.0,1.0,0.0,1.0,0.0,1.028986,2.202899,0.028986,...,1.564058,204.838696,2072.0,6.0,69.0,1.0,6.0,2.0,0.1851,0.189973
3,FEMALE,aa21f31def4edbdcead818afcdfc4d32,1.0,1.0,0.0,0.0,0.0,2.000000,2.000000,0.000000,...,90.900000,143.640000,2054.0,2050.0,1.0,0.0,1.0,1.0,0.0000,0.387567
4,FEMALE,668c6aac52ff54d4828ad379cdb38e7d,1.0,1.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,...,0.000000,0.000000,2053.0,2053.0,1.0,0.0,1.0,1.0,0.0000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46274,FEMALE,5b34391ec6fbc0f189cb8d3d88806199,0.0,1.0,1.0,0.0,0.0,0.400000,0.888889,0.000000,...,39.807333,84.952000,1372.0,50.0,45.0,0.0,7.0,2.0,0.0091,0.352567
46275,FEMALE,198fd2f143f70b149344bcaf7eddee12,1.0,1.0,1.0,0.0,0.0,1.055556,1.055556,0.055556,...,13.367778,76.871111,646.0,124.0,18.0,1.0,2.0,2.0,0.1210,0.209202
46276,FEMALE,338b5c8ade4af1a562d55d4036710630,0.0,1.0,0.0,0.0,0.0,0.181818,0.181818,0.000000,...,0.000000,47.437273,1308.0,998.0,11.0,1.0,2.0,1.0,0.1500,0.150000
46277,FEMALE,2115c065bfc1f3b39e4c87c202e80fa5,1.0,1.0,0.0,0.0,0.0,2.800000,3.000000,0.000000,...,50.990000,142.458000,1410.0,1287.0,5.0,0.0,1.0,2.0,0.1824,0.320760


In [41]:
training_set_path = "../data/processed/training_set.parquet"

training_set.to_parquet(training_set_path)